#Creazione classe dataset

In [1]:
import torch
import numpy as np
import pandas as pd
import fiona
import os

!pip install rasterio

import rasterio

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
percorso =  "/content/drive/MyDrive/Progetto_laboratorio"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 40.0 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
from torch.utils.data import Dataset
class Dataset(Dataset):
  def __init__(self,root,transform = None):
    self.root = root  #directory radice
    self.tranform = transform
    self.zones = self.load_zones()


  def load_zones(self):
    zones = {}       #dizionario dove ad ogni chiave corrispondono
    #itero attraverso le cartelle per trovare percorsi delle immagini
    for zone in os.listdir(self.root):
      zone_path = os.path.join(self.root,zone)
      if os.path.isdir(zone_path):
        zones[zone] = self.load_images_and_labels(zone_path)
    return zones

  def load_images_and_labels(self,zone_path):
    image_label_pairs = []
    if os.path.isdir(zone_path):
      images_dir = os.path.join(element_path, 'images_masked')     #directory con immagini
      labels_dir = os.path.join(element_path, 'labels_match')      #directory con labels
      if os.path.exists(images_dir) and os.path.exists(labels_dir):
        for image_file in os.listdir(images_dir):
          if (image_file.endswith('.tif')):
            image_path = os.path.join(images_dir,image_file)
            label_file = os.path.splitext(image_file)[0] + '.geojson'
            label_path = os.path.join(labels_dir,label_file)

            if os.path.exists(label_path):
              image_label_pairs.append((image_path,label_path))
    return image_label_pairs


  def __len__(self):
    return sum(len(pairs) for pairs in self.zones.values())


  def __getitem__(self,idx):
    image_path = self.image_paths[idx]
    label_path = self.label_paths[idx]



In [4]:
from rasterio.plot import show

with rasterio.open('/content/drive/MyDrive/Progetto_laboratorio/train/train/L15-0331E-1257N_1327_3160_13/images_masked/global_monthly_2018_01_mosaic_L15-0331E-1257N_1327_3160_13.tif') as src:
  image = src.read().astype(np.float32)
  print(image)

[[[170. 145. 144. ...  75.  70.  63.]
  [176. 184. 183. ...  71.  65.  62.]
  [180. 195. 187. ...  65.  63.  68.]
  ...
  [132. 131. 131. ...  82.  60.  49.]
  [119. 120. 120. ...  81.  62.  48.]
  [100. 100. 102. ...  80.  57.  46.]]

 [[148. 126. 125. ...  67.  64.  60.]
  [151. 156. 154. ...  64.  61.  59.]
  [152. 165. 157. ...  60.  59.  61.]
  ...
  [120. 118. 118. ...  74.  61.  55.]
  [110. 109. 110. ...  74.  64.  55.]
  [ 93.  94.  94. ...  74.  61.  53.]]

 [[106.  82.  88. ...  45.  41.  37.]
  [109. 117. 119. ...  43.  39.  37.]
  [116. 134. 127. ...  38.  37.  40.]
  ...
  [ 88.  86.  87. ...  47.  34.  30.]
  [ 80.  80.  80. ...  48.  37.  30.]
  [ 67.  68.  70. ...  48.  34.  28.]]

 [[255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  ...
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]
  [255. 255. 255. ... 255. 255. 255.]]]
